In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

In [2]:
data = load_iris()
X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [3]:
class CustomLogisticRegression:
    def __init__(self, learning_rate=0.01, epochs=1000):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.weights = None
        self.bias = None

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def fit(self, X, y):
        self.classes = np.unique(y)
        self.weights = []
        self.bias = []

        for cls in self.classes:
            y_binary = np.where(y == cls, 1, 0)
            weights = np.zeros(X.shape[1])
            bias = 0

            for _ in range(self.epochs):
                model = np.dot(X, weights) + bias
                predictions = self.sigmoid(model)

                dw = np.dot(X.T, (predictions - y_binary)) / X.shape[0]
                db = np.sum(predictions - y_binary) / X.shape[0]

                weights -= self.learning_rate * dw
                bias -= self.learning_rate * db

            self.weights.append(weights)
            self.bias.append(bias)

    def predict(self, X):
        logits = [np.dot(X, weights) + bias for weights, bias in zip(self.weights, self.bias)]
        return np.argmax(logits, axis=0)

In [4]:
logistic_clf = CustomLogisticRegression()
logistic_clf.fit(X_train, y_train)
logistic_preds = logistic_clf.predict(X_test)

In [6]:
print("Custom Logistic Regression Accuracy:", accuracy_score(y_test, logistic_preds))

Custom Logistic Regression Accuracy: 0.8666666666666667


In [7]:
class CustomKNN:
    def __init__(self, k=3):
        self.k = k

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict(self, X):
        predictions = []
        for x in X:
            distances = np.sqrt(np.sum((self.X_train - x) ** 2, axis=1))
            k_nearest_indices = np.argsort(distances)[:self.k]
            k_nearest_labels = self.y_train[k_nearest_indices]
            predictions.append(np.argmax(np.bincount(k_nearest_labels)))
        return np.array(predictions)

In [8]:
knn_clf = CustomKNN(k=5)
knn_clf.fit(X_train, y_train)
knn_preds = knn_clf.predict(X_test)

In [9]:
print("Custom K-Nearest Neighbors Accuracy:", accuracy_score(y_test, knn_preds))

Custom K-Nearest Neighbors Accuracy: 1.0


In [10]:
class CustomDecisionTree:
    def fit(self, X, y):
        self.tree = self.build_tree(X, y)

    def build_tree(self, X, y, depth=1, max_depth=5):
        if len(set(y)) == 1 or depth == max_depth:
            return np.argmax(np.bincount(y))

        best_split = None
        max_gain = -float("inf")

        for feature in range(X.shape[1]):
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                left_indices = X[:, feature] < threshold
                right_indices = ~left_indices
                gain = self.information_gain(y, left_indices, right_indices)
                if gain > max_gain:
                    max_gain = gain
                    best_split = {"feature": feature, "threshold": threshold, "left": left_indices, "right": right_indices}

        left_tree = self.build_tree(X[best_split["left"]], y[best_split["left"]], depth + 1)
        right_tree = self.build_tree(X[best_split["right"]], y[best_split["right"]], depth + 1)

        return {"split": best_split, "left": left_tree, "right": right_tree}

    def predict_one(self, x, tree):
        if isinstance(tree, dict):
            if x[tree["split"]["feature"]] < tree["split"]["threshold"]:
                return self.predict_one(x, tree["left"])
            else:
                return self.predict_one(x, tree["right"])
        else:
            return tree

    def predict(self, X):
        return np.array([self.predict_one(x, self.tree) for x in X])

    def information_gain(self, y, left_indices, right_indices):
        # Calculating Gini impurity gain
        left, right = y[left_indices], y[right_indices]
        p_left, p_right = len(left) / len(y), len(right) / len(y)
        gain = self.gini(y) - (p_left * self.gini(left) + p_right * self.gini(right))
        return gain

    def gini(self, y):
        proportions = [np.sum(y == c) / len(y) for c in np.unique(y)]
        return 1 - sum(p ** 2 for p in proportions)

In [11]:
tree_clf = CustomDecisionTree()
tree_clf.fit(X_train, y_train)
tree_preds = tree_clf.predict(X_test)

In [12]:
print("Custom Decision Tree Accuracy:", accuracy_score(y_test, tree_preds))

Custom Decision Tree Accuracy: 1.0


In [14]:
all_preds = np.array([logistic_preds, knn_preds, tree_preds])
voting_preds = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=all_preds)

print("Voting Classifier Accuracy:", accuracy_score(y_test, voting_preds))

Voting Classifier Accuracy: 1.0


In [15]:
voting_accuracy = accuracy_score(y_test, voting_preds)
print("Combined Voting Classifier Accuracy:", voting_accuracy)

voting_classification_report = classification_report(y_test, voting_preds)
print("\nClassification Report - Combined Voting Classifier:\n", voting_classification_report)

Combined Voting Classifier Accuracy: 1.0

Classification Report - Combined Voting Classifier:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      1.00      1.00        13
           2       1.00      1.00      1.00        13

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45

